In [1]:
# import processing
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import tqdm
import librosa
import scipy
import torch
import torch.nn.functional as F

from scipy.spatial import distance
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
EspeakWrapper.set_library(_ESPEAK_LIBRARY)
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")




Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably

If you could not run above code, you may need install a program called: eSpeak NG

Here is the link [https://github.com/espeak-ng/espeak-ng/releases/download/1.49.2/espeak-ng-x64.msi](https://github.com/espeak-ng/espeak-ng/releases/download/1.49.2/espeak-ng-x64.msi)

![P1](https://dashpulsar.github.io/BrainlikeASR/Wav2vec2.png)

# Wav2Vec2Phoneme
##### Wav2Vec2Phoneme (W2VP) is an Automatic Speech Recognition (ASR) model that is fine-tuned at the phoneme level based on the Wav2vec 2.0 model developed by the Meta AI team. Wav2vec is a self-supervised speech representation model that learns hidden representations of speech by pre-training on a large amount of unlabeled speech data.
##### Since W2VP is entirely based on the Wav2vec model, differing only in the fine-tuning stage with a different approach, this document will first provide a detailed explanation of the structure and principles of the Wav2vec model.
## 1. Basic Structure of Wav2vec 2.0
##### Wav2vec 2.0 (W2V2) is essentially an encoder network. Its core functionality involves generating latent representations through a combination of CNN and Transformer structures. In the fine-tuning stage, it is often paired with a decoder for joint training (the decoder can better map the latent representations to specific discrete representations, such as words, phonemes, or letters). The main components of W2V2 include a multi-layer CNN structure and a Transformer layer, along with several auxiliary components.


In [4]:
# Wav2vec2.0-Phoneme model structure
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [7]:
# Wav2vec2.0 encoder part model structure
model.wav2vec2

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout)

### 1.1 Feature Encoder (also called Feature Extractor)
##### The feature extractor consists of a seven-layer convolutional neural network (CNN) responsible for extracting low-level features from the raw speech waveform (typically 16kHz sampled audio). This module converts continuous speech input into a series of more compact feature representations, reducing the data's dimensionality while capturing the temporal structure of the speech.
##### More specifically, you can check the code block outputs in the Jupyter notebook for details on the structure. There are seven layers in total (layers 0-6), where the structure of the first layer, layers 1-4, and layers 5-6 are consistent within their respective groups. Why is it designed this way? The original paper does not provide a detailed explanation, but I can offer my interpretation.


In [8]:
# CNNs layer structure
model.wav2vec2.feature_extractor

Wav2Vec2FeatureEncoder(
  (conv_layers): ModuleList(
    (0): Wav2Vec2LayerNormConvLayer(
      (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (activation): GELUActivation()
    )
    (1-4): 4 x Wav2Vec2LayerNormConvLayer(
      (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (activation): GELUActivation()
    )
    (5-6): 2 x Wav2Vec2LayerNormConvLayer(
      (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (activation): GELUActivation()
    )
  )
)

##### For the first layer, the input data is the raw waveform, a PCM digital audio recording at a sampling rate of 16,000 Hz (one-dimensional data where each 16,000 units represent 1 second of recording, with each element being an amplitude value ranging from -1 to 1). We can test the output in the code block to observe how it changes with different inputs. Assuming the input is a random 1-second audio clip (shape (1,16000)), we get an output of shape (512, 3199).


In [88]:
# for the 1st layer

random_input = torch.tensor(np.random.rand(1,16000)).float()

# change input size to 16005 by remove '#'of below code
#random_input = torch.tensor(np.random.rand(1,16005)).float()
output = model.wav2vec2.feature_extractor.conv_layers[0].conv(random_input)

"""
Why output is 512,3199?

Basicly it is depands on the parameter setting of the CNN layer. 
Output length = (input length - kernel size) / stride + 1
              = here (16000-10) /5 +1
              = 3199

you could simply get 3200 by change the input size to 16005

"""
print(output.shape)



torch.Size([512, 3199])


##### Why 512? This process is somewhat analogous to the choice of the number of mel filters in traditional mel spectrogram methods. A mel spectrogram uses the Fast Fourier Transform (FFT) to convert a raw waveform from a time-domain representation to a frequency-domain representation. Previous research found that humans do not perceive changes in frequency linearly; in other words, people cannot easily detect small changes in higher frequencies (e.g., from 1000 Hz to 1020 Hz). Therefore, 512 triangular filters (mel filter banks) are typically set up across different frequency bands within the range of 0-8192 Hz to perform the time-to-frequency domain conversion. This traditional method effectively captures the non-linear characteristics of human auditory perception. However, in the field of deep learning, researchers have discovered that changes between the time and frequency domains do not significantly impact model recognition performance. Thus, instead of preprocessing the raw audio, 512 CNN kernels are used to transform the raw audio into 512 different representations. Since the CNN parameters are learnable, it was initially hoped that the model would automatically learn a filter distribution similar to the mel filter bank (although this is not always the case).


In [87]:
random_input = torch.tensor(np.random.rand(1,16000)).float()
output_CNN = model.wav2vec2.feature_extractor(random_input)
print(output_CNN.shape)

torch.Size([1, 512, 49])


##### Each convolution operation calculates the convolution values of the corresponding elements within the current kernel window, effectively compressing and activating the data. Since the primary function of the first CNN layer is to compress and expand the representation of the audio, smaller kernel sizes and strides are used in subsequent layers to prevent information loss. As a result, each convolution operation in the first and middle layers partially overlaps; in other words, each element carries some information from the previous element (50% overlap in the first layer, 33% in the middle layers). For the fifth and sixth layers, a kernel size of 2 is used to prevent excessive aggregation of edge information as the depth increases. Thus, in the final two layers, each convolution does not carry over information from the previous element.
##### The final output is a tensor of shape (1, 512, N), where N depends on the input length (if the input is 16,000, N will be 49). The model then passes this latent feature representation to a projection layer, which uses a linear transformation to adjust the 512-dimensional input to 1024 dimensions for the next layer. This step primarily adjusts the shape of the original input to match the hidden layer dimensions of the Transformer, as prior research indicates that the hidden layer size directly affects the model's capacity and expressiveness. A Transformer with larger hidden layers possesses better expressiveness and generalization ability, capturing more fine-grained features.


### 1.2 Transformer Layer (Encoder Layer)
##### The Transformer layer consists of 24 standard Scaled Dot-Product Attention (SDPA) layers stacked together. For more details on SDPA, refer to this link. Here, the Q (Query), K (Key), and V (Value) matrices are obtained by multiplying the output from the CNN and projection layers with three learnable weight matrices (each of size 1024x1024). Q, K, and V represent the Query, Key, and Value, respectively. To put it simply, this is similar to entering a query into a search engine, where the query corresponds to Q, which is matched against certain keys (K) to retrieve the desired values (V). During training, the input is distributed among Q, K, and V, while during inference, only the Query (Q) is provided.



In [99]:
model.wav2vec2.encoder.layers

ModuleList(
  (0-23): 24 x Wav2Vec2EncoderLayerStableLayerNorm(
    (attention): Wav2Vec2SdpaAttention(
      (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
      (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (dropout): Dropout(p=0.1, inplace=False)
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (feed_forward): Wav2Vec2FeedForward(
      (intermediate_dropout): Dropout(p=0.0, inplace=False)
      (intermediate_dense): Linear(in_features=1024, out_features=4096, bias=True)
      (intermediate_act_fn): GELUActivation()
      (output_dense): Linear(in_features=4096, out_features=1024, bias=True)
      (output_dropout): Dropout(p=0.1, inplace=False)
    )
    (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
)

##### Through this method, data at each time step can influence and be influenced by other time steps, allowing the model to understand global dependencies within a sequence as training progresses. Thus, the output of these 24 Transformer layers is a feature value that is highly integrated with contextual relationships. However, since the output of our fine-tuning process is time-series data (a sequence of phonemes from a speech segment), the elements of the model's output will retain their original positional information from the CNN layer's output. In other words, if we obtain an output with dimensions (1024, 49), these 49 vectors still preserve their characteristic representation of their original positions in the raw speech, while also reflecting the contextual expression of the sequence. This is inherently a black-box process, but the results can be better understood through the output, as shown in the code block below.


In [120]:
import requests
from io import BytesIO
from pydub import AudioSegment
from pydub.playback import play
speech1="https://dashpulsar.github.io/BrainlikeASR/Wav_000_A_BOY_FELL_FROM_A_WINDOW.wav"
speech2="https://dashpulsar.github.io/BrainlikeASR/Wav_014_HE_GREW_LOTS_OF_VEGETABLES.wav"
response1 = requests.get(speech1)
response2 = requests.get(speech2)
if response1.status_code == 200:
    audio_file1 = BytesIO(response1.content)
    audio_data1, sr = librosa.load(audio_file1)
    wave_res1 = librosa.resample(audio_data1, orig_sr=sr, target_sr=16000)
if response2.status_code == 200:
    audio_file2 = BytesIO(response2.content)
    audio_data2, sr = librosa.load(audio_file2)
    wave_res2 = librosa.resample(audio_data2, orig_sr=sr, target_sr=16000)

In [130]:
"""
If we input the speech file Wav_000_A_BOY_FELL_FROM_A_WINDOW.wav
from https://dashpulsar.github.io/BrainlikeASR/Wav_000_A_BOY_FELL_FROM_A_WINDOW.wav
we could get an output from the wav2vec2-phoneme, with a tensor.shape ([1, 67, 392])
Among them, 67 is the length of speech after compression from CNNs (feature_extractor),
392 is the dimension of the Transformers' projection layer output, 
which represent the expression strength of the phoneme category. 
Here you may ask English language normally has around 60 phoneme, 
because when this model was trained, the input data was speech data from various regions, 
so it contained other phonemes. But our Decoder was limited to English expression,
only 60 of the phoneme categories would be strongly expressed.

"""

input=processor(wave_res1,sampling_rate=16000, return_tensors="pt").input_values
with torch.no_grad():
    out_wav2vec1=model(input).logits

print(out_wav2vec1.shape)

torch.Size([1, 67, 392])


##### The output dimension of the Transformer layer is 1024, and we can modify this dimension through a projection layer to 392, making the representation more logically meaningful. Ultimately, we obtain a 392-dimensional phoneme representation that retains the original time steps; in the code example, the time length is 67. We can directly map the index of the maximum value within each 392-dimensional vector in the matrix to find the corresponding phoneme expression in our tokenizer's vocabulary. Some noise data, such as <pad>, repeated labels, or elongated phonemes, may be present. We can use a Connectionist Temporal Classification (CTC)-based decoder to obtain a calibrated phoneme sequence.

In [135]:
"""

"""
vocab=processor.tokenizer.get_vocab()
vocab_dict=zip(vocab.values(),vocab.keys())
vocab_dict=dict(vocab_dict)
outinds=torch.argmax(out_wav2vec1,dim=-1).numpy()
phoneme_out=[]
for _,i in enumerate(outinds[0]):
    phoneme_out.append(vocab_dict[i])
print(phoneme_out)

['<pad>', '<pad>', '<pad>', '<pad>', 'ɐ', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'b', '<pad>', '<pad>', 'oɪ', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'f', '<pad>', '<pad>', 'ɛ', '<pad>', 'l', '<pad>', '<pad>', '<pad>', 'f', 'ɹ', 'ɹ', 'ʌ', '<pad>', '<pad>', '<pad>', 'm', '<pad>', '<pad>', 'ð', 'ə', '<pad>', '<pad>', '<pad>', 'w', '<pad>', 'ɪ', '<pad>', '<pad>', '<pad>', 'n', 'n', '<pad>', '<pad>', 'd', '<pad>', 'oʊ', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [134]:
transcription = processor.batch_decode(outinds)
print(transcription)

['ɐ b oɪ f ɛ l f ɹ ʌ m ð ə w ɪ n d oʊ']


## 2. Wav2Vec2 Training Process
##### The core idea of Wav2Vec2 is to learn and fit speech features to a certain representation through a two-stage training process. In the self-supervised learning phase, the model learns to discretize audio data and obtain richer latent representations without labeled data. After this, it proceeds to the next stage.




#### Vector Quantization (VQ):
##### During the self-supervised learning phase, the output of the CNN layers is paired with a codebook used for Vector Quantization (VQ). The codebook is randomly initialized and remains fixed during training. It consists of two separate codebooks, each containing 320 entries, with each entry being a 128-dimensional vector. These codewords are designed to match the vectors output by the CNN, effectively mapping the continuous features into a discrete representational space. This process is somewhat analogous to the embedding process in NLP tasks, but unlike textual data, which is inherently discrete, speech data is continuous.
##### In this setup, the latent representations produced by the CNNs are masked during training, then matched with the codebook, replacing the original vectors with fixed codewords. This helps the model learn a more discrete representation of the speech data. The key idea is to force the model to use a limited set of discrete tokens (codewords) to represent a continuous input, which can significantly improve robustness and generalization.


#### Masking:
##### The purpose of masking is to teach the model how to reconstruct masked parts of the speech signal by relying on contextual information. This strategy is similar to the Masked Language Model (MLM) used in BERT for natural language processing but is applied here to speech signals. The masking occurs after the CNN layers and before the Transformer layers. It involves masking portions of the continuous speech data, which the Transformer layers then attempt to predict. By randomly masking parts of the features, the model is forced to learn both global context and local features, thereby capturing more representative speech representations.


#### Learning Functions:
##### The training process involves a combination of two loss functions: Contrastive Loss and Diversity Loss.
##### Contrastive Loss: This is essentially a cross-entropy loss that measures how well the model can predict the correct codeword for a masked input from a set of negative samples. For a given masked position, the model computes a similarity score between the true masked codeword and other codewords in the codebook. The loss is defined as:

$$
L_{\text{contrastive}} = -\log \frac{\exp(\text{sim}(z, c^+))}{\sum_{c \in C} \exp(\text{sim}(z, c))}
$$

##### where z is the masked feature vector, c+ is the correct codeword, and C is the set of all codewords in the codebook. The similarity function sim(⋅)measures how close the masked vector is to each codeword, typically using cosine similarity.




##### Diversity Loss: The goal of this loss is to maximise the utilisation of the codebook entries, encouraging the model to use as many different codewords as possible. This ensures that the model leverages the full expressive capacity of the discrete representational space. The Diversity Loss is defined as:

$$
L_{\text{diversity}} = \frac{1}{|G|} \sum_{g \in G} H\left(\frac{1}{|V|} \sum_{v \in V} \text{softmax}(q_{g,v})\right)

$$

##### where H(⋅) represents entropy, G is the set of codebooks, V is the set of entries in a codebook, and qg,v​ represents the likelihood of choosing entry v from codebook g. The higher the entropy, the more uniformly the entries are utilised, promoting diversity in codeword usage.
##### Through the self-supervised learning process outlined above, Wav2Vec2 develops a highly rich encoder representation. After the self-supervised phase, the masking and codebook components can be disabled, and the model can be fine-tuned in a supervised manner to achieve more task-specific representations.


## 3. Fine-Tuning in Wav2Vec2
##### The fine-tuning phase of Wav2Vec2 involves training the model on labelled data to specialise its learned representations for a specific downstream task, such as Automatic Speech Recognition (ASR). This phase follows the self-supervised pre-training stage, where the model learns to capture robust and generalizable speech features from vast amounts of unlabeled audio data. Here, we focus on how Wav2Vec2 is fine-tuned to enhance its performance in recognizing phonemes or words accurately.

##### In the case of Wav2Vec2Phoneme (W2VP), the output of Wav2Vec2 is first projected to a 392-dimensional space using a linear layer, and then combined with a decoder for supervised learning with labelled data. The decoder here uses a Connectionist Temporal Classification (CTC) model, which is well-suited to capture repeated labels across time and filter out irrelevant tokens such as <pad>. The result can be observed in the code block above.


### 3.1 Objective of Fine-Tuning
##### During fine-tuning, the primary objective is to adapt the pre-trained model’s representations to the specific requirements of the target task. For ASR tasks, this involves mapping the learned speech features to their corresponding textual representations, such as phonemes, letters, or words. Fine-tuning enables the model to become more sensitive to task-specific nuances, such as different accents, phoneme articulations, or speaker variations, by learning from labelled data that directly correlates audio inputs with their textual labels.


### 3.2 Fine-Tuning Process
#### 3.2.1 Initialization with Pre-Trained Weights:
##### Fine-tuning starts by initializing the model with the pre-trained weights obtained from the self-supervised learning phase. These weights contain rich representations of speech features learned from massive unlabeled datasets. The pre-trained weights provide a strong foundation, allowing the model to achieve high performance even with relatively smaller amounts of labeled data.


#### 3.2.2 Linear Projection Layer:
##### Before fine-tuning, a linear projection layer is typically added to transform the output of the Transformer layers to a lower-dimensional space that matches the number of target classes. For example, in the Wav2Vec2Phoneme (W2VP) model, the 1024-dimensional output from the Transformer layers is projected down to 392 dimensions to correspond to the number of possible phoneme classes. This linear transformation aligns the feature space with the task-specific output space, making it easier for the model to learn the mapping from speech features to target labels.
#### 3.2.3 Connectionist Temporal Classification (CTC) Loss:
##### The fine-tuning process employs the Connectionist Temporal Classification (CTC) loss function, which is particularly well-suited for sequence-to-sequence tasks like speech recognition, where the input (audio signal) and output (text labels) may have different lengths. CTC allows the model to learn alignments between the input speech frames and the output labels without requiring frame-level annotations.
##### The CTC loss function calculates the negative log probability of the correct output sequence given the model’s predictions over all possible alignments of the input and output sequences. This makes it ideal for handling variable-length input-output pairs, accounting for uncertainties in the alignment of speech and text, such as pauses, prolonged sounds, or repeated characters.
##### Mathematically, CTC loss is defined as:
$$
L_{\text{CTC}} = -\log p(y \mid X)
$$


##### where X is the input sequence, and y is the target output sequence. The probability p(y∣X) is computed by summing over all possible alignments between X and y.